In [1]:
from pyspark.sql import SparkSession

# 메모리 용량 설정
MAX_MEMORY = '4g'

spark = SparkSession.builder.appName("movie-recommendation")\
            .config("spark.executor.memory", MAX_MEMORY)\
            .config("spark.driver.memory", MAX_MEMORY)\
            .getOrCreate()

spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/01 10:09:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/01 10:09:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
filepath = "/home/ubuntu/working/spark-examples/data/ratings.csv"
ratings_df = spark.read.csv(f"file:///{filepath}", inferSchema=True, header=True)
ratings_df.show()

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|    296|   5.0|1147880044|
|     1|    306|   3.5|1147868817|
|     1|    307|   5.0|1147868828|
|     1|    665|   5.0|1147878820|
|     1|    899|   3.5|1147868510|
|     1|   1088|   4.0|1147868495|
|     1|   1175|   3.5|1147868826|
|     1|   1217|   3.5|1147878326|
|     1|   1237|   5.0|1147868839|
|     1|   1250|   4.0|1147868414|
|     1|   1260|   3.5|1147877857|
|     1|   1653|   4.0|1147868097|
|     1|   2011|   2.5|1147868079|
|     1|   2012|   2.5|1147868068|
|     1|   2068|   2.5|1147869044|
|     1|   2161|   3.5|1147868609|
|     1|   2351|   4.5|1147877957|
|     1|   2573|   4.0|1147878923|
|     1|   2632|   5.0|1147878248|
|     1|   2692|   5.0|1147869100|
+------+-------+------+----------+
only showing top 20 rows



In [3]:
# timestamp는 빼고 선택
import pyspark.sql.functions as F

ratings_df = ratings_df.select(F.col("userId"), F.col("movieId"), F.col("rating"))
ratings_df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



In [4]:
# 통계 정보 확인
ratings_df.select(F.col("rating")).describe().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|          25000095|
|   mean| 3.533854451353085|
| stddev|1.0607439611423535|
|    min|               0.5|
|    max|               5.0|
+-------+------------------+



In [5]:
# 훈련 세트와 테스트 세트 분리
train_df, test_df = ratings_df.randomSplit([0.8, 0.2], seed=42)

In [6]:
# 모델 생성
from pyspark.ml.recommendation import ALS

als = ALS(
    maxIter=5, # 훈련 횟수
    regParam=0.1, # 가중치 정규화 계수
    
    userCol = "userId", # 사용자 정보 컬럼
    itemCol = "movieId", # 아이템 컬럼(여기서는 영화)
    ratingCol = "rating", # 평점 컬럼
    
    coldStartStrategy='drop' # 학습하지 못한 데이터에 대한 처리. 여기서는 삭제
)

In [7]:
# 모델을 훈련하면 모델 객체가 등장
als_model = als.fit(train_df)

23/08/01 10:44:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/08/01 10:44:36 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [8]:
print(type(als))
print(type(als_model))

<class 'pyspark.ml.recommendation.ALS'>
<class 'pyspark.ml.recommendation.ALSModel'>


In [9]:
# 예측을 할 때 사이킷 런은 predict 메소드를 사용하지만
# 스파크에서는 feature가 들어가서 label 나오는 데이터의 변환으로 판단 - transform

predictions = als_model.transform(test_df)
predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|     1|    307|   5.0|  4.128557|
|     1|   1175|   3.5|  3.946729|
|     1|   1237|   5.0|   4.04257|
|     1|   2012|   2.5| 3.0258772|
|     1|   2692|   5.0| 3.9533136|
|     1|   3949|   5.0|  3.941097|
|     1|   4973|   4.5| 4.1592965|
|     1|   5912|   3.0| 3.8545005|
|     1|   7318|   2.0| 2.7808855|
|     1|   7323|   3.5| 3.8591025|
|     1|   7327|   3.5| 3.9855494|
|     1|   7365|   4.0| 3.6865401|
|     1|   7937|   3.0|  3.793906|
|     1|   8014|   3.5|   3.98544|
|     1|   8786|   4.0| 3.8284254|
|     1|  32591|   5.0| 3.4565766|
|     3|      1|   4.0|  3.939362|
|     3|    111|   4.0| 3.9599946|
|     3|    214|   5.0| 3.8286722|
|     3|    293|   5.0|  4.306011|
+------+-------+------+----------+
only showing top 20 rows



In [15]:
# 평가(RMSE)
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

In [16]:
rmse = evaluator.evaluate(predictions)
rmse

0.8107066196875714

In [17]:
# 추천하기 1
#  각 user에게 top 3 아이템(영화) 추천
als_model.recommendForAllUsers(3).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{177209, 5.61151...|
|     3|[{203882, 6.20100...|
|     5|[{177209, 6.21581...|
|     6|[{177209, 6.25269...|
|     9|[{177209, 6.35110...|
|    12|[{177209, 5.41620...|
|    13|[{177209, 6.12787...|
|    15|[{194434, 6.99622...|
|    16|[{194434, 6.26107...|
|    17|[{127252, 5.54872...|
|    19|[{194434, 5.56703...|
|    20|[{177209, 6.78249...|
|    22|[{204990, 6.40798...|
|    26|[{177209, 5.48991...|
|    27|[{203882, 6.20889...|
|    28|[{194434, 7.35356...|
|    31|[{177209, 3.94370...|
|    34|[{177209, 5.56502...|
|    35|[{203882, 6.15724...|
|    37|[{194434, 6.38102...|
+------+--------------------+
only showing top 20 rows



In [18]:
# 추천하기 2
#  각 영화에 어울리는 top 3 유저를 추천
als_model.recommendForAllItems(3).show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|      1|[{18230, 5.656654...|
|      3|[{87426, 5.410471...|
|      5|[{143282, 5.53671...|
|      6|[{156318, 5.71069...|
|      9|[{87426, 5.070337...|
|     12|[{87426, 5.459793...|
|     13|[{87426, 5.444057...|
|     15|[{87426, 5.135165...|
|     16|[{156318, 5.62265...|
|     17|[{58248, 5.673537...|
|     19|[{87426, 5.278235...|
|     20|[{87426, 5.016489...|
|     22|[{52924, 5.354927...|
|     26|[{93907, 5.208003...|
|     27|[{87426, 5.657512...|
|     28|[{58248, 5.649009...|
|     31|[{143282, 5.45220...|
|     34|[{141970, 5.55717...|
|     35|[{86709, 5.01046}...|
|     37|[{7629, 5.052722}...|
+-------+--------------------+
only showing top 20 rows



In [21]:
# 사용자 목록을 만들어서 추천. 반드시 dataframe 으로 만들어서 예측
from pyspark.sql.types import IntegerType

user_list = [276, 53, 393]
users_df = spark.createDataFrame(user_list, IntegerType()).toDF("userId")
users_df.show()

+------+
|userId|
+------+
|   276|
|    53|
|   393|
+------+



In [22]:
user_recommend = als_model.recommendForUserSubset(users_df, 5)
user_recommend.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|    53|[{194334, 6.74142...|
|   276|[{177209, 7.08180...|
|   393|[{203086, 6.66976...|
+------+--------------------+



In [23]:
# 특정 user를 위한 추천
movies_list = user_recommend.filter("userId = 393").select("recommendations")
movies_list.show()

+--------------------+
|     recommendations|
+--------------------+
|[{203086, 6.66976...|
+--------------------+



In [24]:
# 데이터 프레임으로 조회해서 가져오는 것이 아닌, 실제 데이터를 가져와야 하기 때문에 collect()를 사용하는 것이 좋다.
movies_list = movies_list.select("recommendations").collect()
movies_list

[Row(recommendations=[Row(movieId=203086, rating=6.669764518737793), Row(movieId=127252, rating=6.586252212524414), Row(movieId=203882, rating=6.54063081741333), Row(movieId=190707, rating=6.16048526763916), Row(movieId=176597, rating=6.053582191467285)])]

In [25]:
movies_list_first = movies_list[0].recommendations
movies_list_first

[Row(movieId=203086, rating=6.669764518737793),
 Row(movieId=127252, rating=6.586252212524414),
 Row(movieId=203882, rating=6.54063081741333),
 Row(movieId=190707, rating=6.16048526763916),
 Row(movieId=176597, rating=6.053582191467285)]

In [26]:
# 추천 테이블
recommend_df = spark.createDataFrame(movies_list_first)
recommend_df.show()

+-------+-----------------+
|movieId|           rating|
+-------+-----------------+
| 203086|6.669764518737793|
| 127252|6.586252212524414|
| 203882| 6.54063081741333|
| 190707| 6.16048526763916|
| 176597|6.053582191467285|
+-------+-----------------+



In [27]:
# 영화 메타 데이터 데이터프레임 만들기
movie_filepath = "/home/ubuntu/working/spark-examples/data/movies.csv"
movies_df = spark.read.csv(f"file:///{movie_filepath}", inferSchema=True, header=True)
movies_df.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [28]:
recommend_df.createOrReplaceTempView("rec")
movies_df.createOrReplaceTempView("movies")

In [29]:
query = """
select *
from movies
join rec on movies.movieId = rec.movieId
order by rating desc
"""

spark.sql(query).show()

+-------+--------------------+--------------------+-------+-----------------+
|movieId|               title|              genres|movieId|           rating|
+-------+--------------------+--------------------+-------+-----------------+
| 203086|Truth and Justice...|               Drama| 203086|6.669764518737793|
| 127252|The Veil of Twili...|Crime|Fantasy|Mys...| 127252|6.586252212524414|
| 203882|Dead in the Water...|              Horror| 203882| 6.54063081741333|
| 190707|         1968 (2018)|  (no genres listed)| 190707| 6.16048526763916|
| 176597|Täällä Pohjantähd...|  (no genres listed)| 176597|6.053582191467285|
+-------+--------------------+--------------------+-------+-----------------+



In [30]:
spark.stop()